In [5]:
import numpy as np
import pandas as pd
import requests

{'adult': False, 'backdrop_path': '/2DbSpBgjHASSOXe5eHC9qVtif0F.jpg', 'belongs_to_collection': {'id': 705370, 'name': 'Beauty Salon: Special Service Series', 'poster_path': None, 'backdrop_path': None}, 'budget': 0, 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}], 'homepage': '', 'id': 452251, 'imdb_id': '', 'original_language': 'ko', 'original_title': '미용실 : 특별한 서비스', 'overview': 'A legendary beauty salon that is sure to become a regular once you go!”Sir, would you like to have a very hot service?”In order to overcome the financial difficulties of the beauty parlor,It begins to attract the male guests with the special technique that makes it from head to toe.Kurt, Firm, and ‘it’ to open the hair salon is good!', 'popularity': 138.219, 'poster_path': '/z3Vse8jRnmKJyZFK6VSw1G6jHnY.jpg', 'production_companies': [], 'production_countries': [{'iso_3166_1': 'KR', 'name': 'South Korea'}], 'release_date': '2016-12-29', 'revenue': 0, 'runtime': 81, 'spoken_languages': [{'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}], 'status': 'Released', 'tagline': '', 'title': 'Beauty Salon: Special Service', 'video': False, 'vote_average': 6.2, 'vote_count': 11}

In [29]:
MAX = 10

API_KEY = "48e0bf10bd721602a930738080e4daf4"
url = f"https://api.themoviedb.org/3/movie/{{}}?api_key={API_KEY}&append_to_response=budget,revenue"

all_movies = []

page_number = 1
while page_number < MAX:
    response = requests.get(f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}&sort_by=popularity.desc&page={page_number}")
    if response.status_code == 200:
        data = response.json()
        for movie in data['results']:
            movie_id = movie['id']
            movie_response = requests.get(url.format(movie_id))
            if movie_response.status_code == 200:
                movie_data = movie_response.json()
                cast_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}")
                if cast_response.status_code == 200:
                    cast_data = cast_response.json()
                    all_movies.append({
                        'Title': movie_data['title'],
                        'Genres': [genre['name'] for genre in movie_data['genres']],
                        'Cast': [actor['name'] for actor in cast_data['cast']][:5],
                        'Budget': movie_data['budget'],
                        'Revenue': movie_data['revenue'],
                        'Popularity': movie_data['popularity'],
                        'Vote_average': movie_data['vote_average'],
                        'Adult' : movie_data["adult"]
                    })
                else:
                    print(f"Error {cast_response.status_code}: {cast_response.reason}")
                    break
            else:
                print(f"Error {movie_response.status_code}: {movie_response.reason}")
                break
        total_pages = data['total_pages']
        if page_number >= total_pages:
            break
        page_number += 1
    else:
        print(f"Error {response.status_code}: {response.reason}")
        break

df = pd.DataFrame(all_movies)
df.head()

,Title,Genres,Cast,Budget,Revenue,Popularity,Vote_average,Adult
0,The Super Mario Bros. Movie,"[Animation, Adventure, Family, Fantasy, Comedy]","[Chris Pratt, Anya Taylor-Joy, Charlie Day, Ja...",100000000,470000000,10312.202,7.554,False
1,Shazam! Fury of the Gods,"[Action, Comedy, Fantasy]","[Zachary Levi, Asher Angel, Jack Dylan Grazer,...",125000000,120864255,8477.075,6.992,False
2,Avatar: The Way of Water,"[Science Fiction, Adventure, Action]","[Sam Worthington, Zoe Saldaña, Sigourney Weave...",350000000,2312335665,4942.864,7.740,False
3,Creed III,"[Drama, Action]","[Michael B. Jordan, Tessa Thompson, Jonathan M...",75000000,269000000,4671.089,7.274,False
4,65,"[Science Fiction, Adventure, Thriller, Action]","[Adam Driver, Ariana Greenblatt, Chloe Coleman...",45000000,55000000,4006.391,6.310,False
